In [1]:
import os
import glob
import numpy as np
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.preprocessing import image

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions


In [2]:
train_data_path = os.path.join('..', 'Dataset', 'Binary_class_dataset', 'train') # '/content/data/train'
valid_data_path = os.path.join('..', 'Dataset', 'Binary_class_dataset', 'test') # '/content/data/test'

train_data_agumentation = ImageDataGenerator(rescale = 1./255,
                                             zoom_range = 0.1
                                             )

val_data_agumentation = ImageDataGenerator(rescale = 1./255)


# load training data
train_data = train_data_agumentation.flow_from_directory(directory = train_data_path,
                                                         target_size = (224,224),
                                                         class_mode = 'categorical',
                                                         batch_size = 8)

val_data = val_data_agumentation.flow_from_directory(directory = valid_data_path,
                                                     target_size = (224,224),
                                                     class_mode = 'categorical',
                                                     batch_size = 8)

Found 10798 images belonging to 2 classes.
Found 7200 images belonging to 2 classes.


In [3]:
checkpoint_dir = 'checkpoints_Binary_class_vgg16_adam'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

checkpoint_filepath = os.path.join(checkpoint_dir,
                                   "model_epoch_{epoch:02d}_val_acc_{val_accuracy:.2f}_val_loss_{val_loss:.2f}.keras")

checkpoint = ModelCheckpoint(filepath = checkpoint_filepath,
                             monitor = 'val_loss',
                             verbose = 0,
                             save_best_only = True,
                             save_weights_only = False,
                             mode = 'auto')

early = EarlyStopping(monitor = 'val_loss',
                      min_delta = 0,
                      patience = 10,
                      verbose = 0,
                      mode = 'auto')

reduceLR = ReduceLROnPlateau(monitor = "val_loss", 
                             factor = 0.1,
                             patience = 5, 
                             verbose = 0, 
                             mode = "auto",
                             min_delta = 0.0001, 
                             cooldown = 0,
                             min_lr = 0.0)

csv_logger = CSVLogger(os.path.join(checkpoint_dir, 'training.log'))

callbacks_list = [checkpoint, reduceLR, csv_logger] #, early


In [4]:
image_size = [224, 224]
num_classes = len(glob.glob(train_data_path+'/*'))

# Load the pre-trained model
model = VGG16(input_shape = image_size + [3],
              weights = 'imagenet',
              include_top = False)

# our layers - you can add more if you want
x = Flatten()(model.output)

output_layer = Dense(num_classes, activation='softmax')(x)

# model.layers.trainable = False
for layer in model.layers:
    layer.trainable = False

# create a model object
model = Model(inputs = model.input, outputs = output_layer)
model.summary()




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [5]:
#Initializing the hyperparameters
adam = Adam(learning_rate = 0.001, 
            beta_1 = 0.9,
            beta_2 = 0.999,
            epsilon = 1e-07)

# compile the model with adam optimizer, categorical_croosentropy loss function
# model.compile(optimizer = 'adam',
#               loss = 'categorical_crossentropy',
#               metrics = ['accuracy'])
model.compile(optimizer = adam,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(train_data,
                    batch_size = 8,
                    epochs = 5,
                    validation_data = val_data,
                    callbacks = callbacks_list)

# steps_per_epoch=len(training_set),
# validation_steps=len(test_set),

# Save the trained model
save_model_path = os.path.join(checkpoint_dir, 'Binary_class_VGG16_adam_model_last_epoch.h5')
model.save(save_model_path)
print(f"Model saved as {save_model_path}.")

Epoch 1/5


1350/1350 [==============================] - 4389s 3s/step - loss: 0.7146 - accuracy: 0.7786 - val_loss: 0.5960 - val_accuracy: 0.8111 - lr: 0.0010
Epoch 2/5
1350/1350 [==============================] - 4716s 3s/step - loss: 0.5771 - accuracy: 0.8242 - val_loss: 0.5235 - val_accuracy: 0.8250 - lr: 0.0010
Epoch 3/5
1350/1350 [==============================] - ETA: 0s - loss: 0.5259 - accuracy: 0.8393

In [ ]:
# accuracies
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.legend()
plt.savefig('Binary_class_VGG16_adam_model_accuracy_and_val_accuracy.png', dpi=200)
plt.show()
 
# loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.savefig('Binary_class_VGG16_adam_model_loss_and_val_loss.png', dpi=200)
plt.show()


In [ ]:
score = model.evaluate(train_data, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

In [ ]:
score = model.evaluate(val_data, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Create Classification Report

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [ ]:
# Create Classification Report

def evaluate_model(y_true, y_pred, class_names=None):
    """
    This function evaluates the performance of a model and prints the confusion matrix, 
    accuracy score, classification report, and Cohen's kappa score. It also plots a heatmap of the confusion matrix.
    
    Args:
        y_true (array-like): Ground truth (true labels)
        y_pred (array-like): Predicted labels from the model
        class_names (list): List of class names for better readability in the confusion matrix
        
    Returns:
        None: Displays the evaluation metrics and heatmap.
    """
    
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Calculate accuracy score
    accuracy = accuracy_score(y_true, y_pred)
    
    # Get classification report
    class_report = classification_report(y_true, y_pred, target_names=class_names)
    
    # Calculate Cohen's kappa score
    kappa_score = cohen_kappa_score(y_true, y_pred)
    
    # Print evaluation metrics
    print("Confusion Matrix:\n", cm)
    print(f"\nAccuracy Score: {accuracy * 100:.2f}%")
    print("\nClassification Report:\n", class_report)
    print(f"Cohen's Kappa Score: {kappa_score:.4f}")
    
    # Plot heatmap for confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix Heatmap')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()


def create_classification_report(model_path, test_data_dir):
    # Step 1: Set up directories and parameters
    # test_data_dir = 'data/train/'  # Path to the directory containing test images in folders 0, 1, 2

    # Step 2: Load the trained model
    model = load_model(model_path)  # Assuming you've saved the model as .h5

    # Step 3: ImageDataGenerator for loading test images (no augmentation needed for testing)
    test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values to [0, 1]

    # Step 4: Load test data using flow_from_directory
    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size = (224, 224),  # Same size as the input size for your model
        batch_size = 32,  # Adjust according to your hardware
        class_mode = 'categorical',  # For multi-class classification
        shuffle = False  # Don't shuffle, we need to keep track of the order for y_true
    )

    # Step 5: Get true labels from the generator
    y_true = test_generator.classes  # These are the true class labels

    # Step 6: Predict using the model
    y_pred_prob = model.predict(test_generator)  # Predict probabilities for each class
    y_pred = np.argmax(y_pred_prob, axis=1)  # Get the index of the max probability (class label)

    # Step 7: Map predicted and true labels to class names
    class_labels = list(test_generator.class_indices.keys())

    # You can use the evaluation function from earlier for further analysis
    evaluate_model(y_true, y_pred, class_names=class_labels)
    
# Call the function

model_path = os.path.join(checkpoint_dir, 'Binary_class_VGG16_adam_model_last_epoch.h5')
test_data_dir = os.path.join('..', 'Dataset', 'Binary_class_dataset', 'val')

create_classification_report(model_path, test_data_dir)